In [232]:
import cv2
import numpy as np

In [255]:
# Yolo 로드
net = cv2.dnn.readNet("yolov4.weights", "yolov4-obj.cfg")
classes = []
with open("objects.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]    # strip : 공백제거
print(len(classes))       # 7
print(classes)

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))
print(colors)

7
['rain', 'wind', 'cloud', 'lightning', 'pool', 'person', 'umbrella']
[[176.74513813  50.79887672 217.69444163]
 [ 92.5850771  198.24688951  25.92496353]
 [ 25.55767939   1.86217304 107.09420734]
 [140.23550885  40.21769786 245.92282652]
 [235.96810203 110.24943147  24.08896413]
 [ 14.84127217   4.10690933   2.2405579 ]
 [178.38174497  94.03945758  78.85078448]]


In [261]:
# 이미지 가져오기
img = cv2.imread("test_data/sample10.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

In [262]:
# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)


In [263]:
# 정보를 화면에 표시
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)


In [264]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [265]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
        
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()